# ETL rent-591 results (json)

In [1]:
import os
import sys
import numpy as np
import json
import re
from pymongo import MongoClient

# self-definition package.module
# del myfun
import quiz.myfun02 as myfun

In [2]:
path_root = os.path.join(os.getcwd(), "../")
path_key = os.path.join(path_root, './key/key.txt')
path_result = os.path.join(path_root, "./result")
json_taipei = os.path.join(path_result, "./results_taipai.json")
json_newtaipei = os.path.join(path_result, "./results_newtaipai.json")

In [3]:
conn_string = open(path_key, 'r').read()

---

## read json

In [4]:
with open(json_taipei, "r", encoding='utf-8') as jsonfile:
    raw_t = json.load(jsonfile)

with open(json_newtaipei, "r", encoding='utf-8') as jsonfile:
    raw_nt = json.load(jsonfile)

---

## Tansform - meta location

In [5]:
# requirement fiels for location 
req_col = {
    'regionid', 'sectionid', 'streetid',
    'region_name', 'section_name', 'street_name',
}

In [6]:
# 每一筆都要做這件事
region = []
street = []
section = []
for i in range(len(raw_t)):      
    req_raw = {k: raw_t[i][k] for k in raw_t[i].keys() & req_col}
    location = myfun.extract_location(req_raw)
    region.append(location['region'])
    street.append(location['street'])
    section.append(location['section'])

for i in range(len(raw_nt)):
    req_raw = {k: raw_nt[i][k] for k in raw_nt[i].keys() & req_col}
    location = myfun.extract_location(req_raw)
    region.append(location['region'])
    street.append(location['street'])
    section.append(location['section'])

In [7]:
# unique list: meta_region, meta_street, meta_section
a = {v['regionid']:v for v in region}.values()
meta_region = [v for v in a]
meta_street = [dict(s) for s in set(frozenset(d.items()) for d in street)]
meta_section = [dict(s) for s in set(frozenset(d.items()) for d in section)]

---

### inert to mongoDB

In [8]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']

In [9]:
collection = db['meta_region']
collection.delete_many({})
collection.insert_many(meta_region)

collection = db['meta_street']
collection.delete_many({})
collection.insert_many(meta_street)

collection = db['meta_section']
collection.delete_many({})
collection.insert_many(meta_section)

In [10]:
cluster.close()

---

## Transform - meta cathay

In [11]:
# requirement fields for cathay 
req_col = {"post_id", "kind","shape","nick_name", "condition",}

In [12]:
# 每一筆都要做這件事
cathay_search = []
for i in range(len(raw_t)):      
    req_raw = {k: raw_t[i][k] for k in raw_t[i].keys() & req_col}
    cathay_search.append(myfun.raw_cathay(req_raw))

for i in range(len(raw_nt)):      
    req_raw = {k: raw_nt[i][k] for k in raw_nt[i].keys() & req_col}
    cathay_search.append(myfun.raw_cathay(req_raw))

---

### inert to mongoDB

In [13]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']

In [14]:
collection = db['cathay_search']
collection.delete_many({})
collection.insert_many(cathay_search)

In [15]:
cluster.close()

---

## Transform - meta my

In [16]:
# requirement fields for me
req_col = {"post_id", "updatetime", "browsenum_all", "hasimg",
           "area", "kind", "shape","room", "floor", "allfloor",
           "regionid","sectionid","streetid",
           "nick_name","price","living","condition",}

In [17]:
# 每一筆都要做這件事
my_search = []
for i in range(len(raw_t)):      
    req_raw = {k: raw_t[i][k] for k in raw_t[i].keys() & req_col}
    my_search.append(myfun.raw_my(req_raw))
    
for i in range(len(raw_nt)):      
    req_raw = {k: raw_nt[i][k] for k in raw_nt[i].keys() & req_col}
    my_search.append(myfun.raw_my(req_raw))

---

### inert to mongoDB

In [18]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']

In [19]:
collection = db['my_search']
collection.delete_many({})
collection.insert_many(my_search)

In [20]:
cluster.close()

---

## Transform - meta data

In [21]:
# 每一筆都要做這件事
# requirement fields for meta
req_col = {"nick_name","post_id","updatetime","refreshtime","browsenum_all",
           "hasimg","cover","photoNum",
           "area","price","unit",
           "kind","shape","houseage",
           "living","condition",
           "room","layout","floorInfo",
           "region_name","section_name","street_name","alley_name","address_img",
          }

In [22]:
# 每一筆都要做這件事
meta_data = []
for i in range(len(raw_t)):      
    req_raw = {k: raw_t[i][k] for k in raw_t[i].keys() & req_col}
    meta_data.append(myfun.raw_meta(req_raw))
    
for i in range(len(raw_nt)):      
    req_raw = {k: raw_nt[i][k] for k in raw_nt[i].keys() & req_col}
    meta_data.append(myfun.raw_meta(req_raw))

---

### inert to mongoDB

In [23]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']

In [24]:
collection = db['meta_data']
collection.delete_many({})
collection.insert_many(meta_data)

In [25]:
cluster.close()

---

## other meta

In [26]:
cluster = MongoClient(conn_string)
db = cluster['db_cathay']

In [27]:

meta_kind = {
    "0": "不限",
    "1": "整層住家",
    "2": "獨立套房",
    "3": "分租套房",
    "4": "雅房",
}

meta_shape = {
    "1": "公寓",
    "2": "電梯大樓",
    "3": "透天厝",
    "4": "別墅",
}

meta_condition = {
    'cook': '可開火',
    'cartplace': '有車位',
    'balcony_0': '沒有陽台',
    'pet': '可養寵物',
    'trabus': '有公車',
    'lease': '可短期租賃',
    'balcony_1': '有陽台',
    'lift': '有電梯',
    'tv': '電視',
    'cold': '冷氣',
    'icebox': '冰箱',
    'hotwater': '熱水器',
    'naturalgas': '天然瓦斯',
    'four': '第四台',
    'broadband': '網路',
    'washer': '洗衣機',
    'bed': '床',
    'wardrobe': '衣櫃',
    'sofa': '沙發',
    'bookTable': '桌子',
    'chair': '椅子',
}

meta_living = {
    'depart': '百貨公司',
    'advstore': '進便利商店',
    'market': '傳統市場',
    'night': '夜市',
    'park': '公園綠地',
    'school': '學校',
    'hospital': '醫療機構',
}

In [28]:
collection = db['meta_kind']
collection.delete_many({})
collection.insert_one(meta_kind)

collection = db['meta_shape']
collection.delete_many({})
collection.insert_one(meta_shape)

collection = db['meta_condition']
collection.delete_many({})
collection.insert_one(meta_condition)

collection = db['meta_living']
collection.delete_many({})
collection.insert_one(meta_living)

In [29]:
cluster.close()

---

## save refresh csv

In [30]:
import pandas as pd

In [31]:
meta_refresh = []
for x in raw_t:
    meta_refresh = meta_refresh + [{'id': x['id'], 'updatetime': x['updatetime']}]
for x in raw_nt:
    meta_refresh = meta_refresh + [{'id': x['id'], 'updatetime': x['updatetime']}]

path_csv = os.path.join(path_result, "./meta_refresh.csv")
pd.DataFrame.from_dict(meta_refresh).to_csv(path_csv, encoding = 'utf-8', index = 0)

## END